# This is a notebook for creating a webscraper for house prices and features.

## Wel'll scrape the data, wrangle it, and then insert it intoa database for later use.


In [ ]:
import time
import re
import string
import json
import unicodedata

import sqlite3
from sqlite3 import Error
from selenium import webdriver

### Classes for crawling the website, scraping the data and finally to insert the data into a relational (SQLite) database

In [86]:
class DataBaseWorker():

    def create_connection(self):
        """ create a database connection to the SQLite database
            specified by db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn = None
        try:
            conn = sqlite3.connect(self.path)
        except Error as e:
            print(e)
        self.conn=conn

    def create_house(self,house_features):
        """
        Create a new project into the projects table
        :param conn:
        :param project:
        :return: project id
        """
        sql = ''' INSERT INTO house_features(HOUSE_TYPE,
                                            HOUSE_ROOMS,
                                            ADDRESS,
                                            AREA,
                                            CITY,
                                            PRICE,
                                            SIZE,
                                            YEAR)
                VALUES(?,?,?,?,?,?,?,?) '''
        cur = self.conn.cursor()
        cur.execute(sql, house_features)
        self.conn.commit()
        return cur.lastrowid


class HouseScraper(DataBaseWorker):
    def __init__(self, city_='Helsinki'):
        self.city = city_
        self.wd = None
        self.max_n = 40
        self.full_list = ['START:']
        self.path = r"../Data/house.db"
        self.conn = None

    def __repr__(self) -> str:
        "A class for scraping house features from the web."

    def search_houses(self):
        """
        Open the connection to the url and search the given city for houses.
        """
        url = 'https://www.etuovi.com/'
        wd = webdriver.Chrome(executable_path=r'../Data/chromedriver.exe')
        wd.get(url)
        time.sleep(5)
        # Accept cookies
        try:
            wd.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/div[3]/button[2]').click()
            time.sleep(2)
        except:
            pass
        # Search for the city
        time.sleep(5)
        wd.find_element_by_id('location').send_keys(self.city)
        time.sleep(2)
        # Click the search button
        wd.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div/div[1]/div/div[2]/div/form/div/div[2]/div[2]/button').click()
        time.sleep(1)
        wd.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div/div[1]/div/div[2]/div/form/div/div[2]/div[2]/button').click()
        time.sleep(5)

        max_n = int(unicodedata.normalize("NFKD", wd.find_elements_by_id('searchResultCount')[0].get_attribute('innerText')).replace(" ", ""))
        if("haku=" in wd.current_url):
            print('Search succesful, found {n} listings.'.format(n=max_n))
        else:
            print("Couldn't search city: {city}".format(city=self.city))

        self.wd = wd
        self.max_n = max_n

    def scrape(self,n=40):
        """
        Starts scraping from the website.
        Params:
        n: maximum number of houses that you want to get
        """
        max_pages = int(min(n,self.max_n)/30) # Either n or at most max_n. 30 results per page
        print("Starting the scraping process...")
        for i in range(max_pages):
            self.wd.find_elements_by_id('searchResultCount')[0].get_attribute('innerText')
            t = self.wd.find_element_by_id('announcement-list').get_attribute('innerText').splitlines()
            self.full_list.extend(t)
            
            time.sleep(3)
            # Scroll to the bottom
            self.wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            # Next page
            self.wd.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div[2]/div[1]/div/div[5]/div[1]/div[7]/button').click()
            if ((i+1)*30)%300 == 0:
                print("Scraped {n_listings}/{max_listings} listings.".format(n_listings=(i+1)*30,max_listings=max_pages*30))
            time.sleep(5)


    def insert_data_to_DB(self):
        i=0
        n_into_DB = 0
        self.create_connection()
        while i < len(self.full_list)-10:
            line = self.full_list[i] # Can't use enumerate because we want to be able to jump forward in the list
            if '|' in line:
                HOUSE_TYPE = re.match('.+?(?=\|)',line).group().strip()
                # print(HOUSE_TYPE)
                if len(HOUSE_TYPE.strip().split()) < 2: # Error spotting in the listings
                    HOUSE_ROOMS = re.search("(?<=\|).*?(?=(?:\()|$)" ,line).group()
                    # Location
                    ADDRESS,*AREA,CITY = [x.strip() for x in self.full_list[i+1].split(",")]
                    if len(AREA) > 1:
                        # There can be multiple AREA names, join them
                        AREA = str("_".join(AREA))
                    AREA = AREA[0] if AREA else ""
                    # Price (rounding to nearest int)
                    try:
                        # PRICE = int(float(unicodedata.normalize("NFKD",self.full_list[i+3]).replace('€',"").replace(" ","").replace(',','.')))
                        PRICE = int(float(re.match('.+?(?=\€)',unicodedata.normalize("NFKD",self.full_list[i+3])).group().replace(" ","").replace(',','.')))
                    except:
                        PRICE = 0
                    # Size (rounding to nearest int)
                    try:
                        SIZE = int(float(unicodedata.normalize("NFKD",self.full_list[i+5].split(" ")[0]).replace(',','.')))
                    except:
                        SIZE = 0
                    # Year
                    try:
                        YEAR = int(self.full_list[i+7].strip())
                    except:
                        YEAR = 0

                    house_1 = (HOUSE_TYPE,HOUSE_ROOMS,ADDRESS,AREA,CITY,PRICE,SIZE,YEAR)
                    self.create_house(house_1)
                    n_into_DB += 1
                    # print('Inserted:',house_1,'into database')

            i+=1
        print("Inserted approximately {n_into_DB} listings to the database.".format(n_into_DB=n_into_DB))
        self.conn.close()

In [87]:
myscraper = HouseScraper(city_='Espoo')
myscraper.search_houses()

Search succesful, found 2415 listings.


In [88]:
myscraper.scrape(n=3000)

Starting the scraping process...
Scraped 300/2400 listings.
Scraped 600/2400 listings.
Scraped 900/2400 listings.
Scraped 1200/2400 listings.
Scraped 1500/2400 listings.
Scraped 1800/2400 listings.
Scraped 2100/2400 listings.
Scraped 2400/2400 listings.


In [89]:
myscraper.insert_data_to_DB()

Inserted approximately 2397 listings to the database.


### For testing purposes below

In [97]:
# # Testing purposes
# t = myscraper.full_list
# i=0
# while i < len(t) :
#     line = t[i] # Can't use enumerate because we want to be able to jump forward in the list
#     # print(line)
#     # if len(line.strip().split()) < 3:
#     if '|' in line:
#         HOUSE_TYPE = re.match('.+?(?=\|)',line).group().strip()
#         if len(HOUSE_TYPE.strip().split()) < 2:
#             # print(len(HOUSE_TYPE.strip().split()))
#             HOUSE_ROOMS = re.search("(?<=\|).*?(?=(?:\()|$)" ,line).group()
#             # print(HOUSE_ROOMS)
#             # Location
#             # print(t[i+1])
#             # print(line)
#             # print(t[i+1].split(","))
#             ADDRESS,*AREA,CITY = [x.strip() for x in t[i+1].split(",")]

#             if len(AREA) > 1:
#                 # There can be multiple AREA names, join them
#                 AREA = str("_".join(AREA))

#             AREA = AREA[0] if AREA else ""

#             # print(ADDRESS)
#             # print(AREA)

#             # Price (rounding to nearest int)
#             try:
#                 PRICE = int(float(unicodedata.normalize("NFKD",t[i+3]).replace('€',"").replace(" ","").replace(',','.')))
#             except:
#                 PRICE = 0
#             # print(PRICE)
#             # Size (rounding to nearest int)
#             try:
#                 SIZE = int(float(unicodedata.normalize("NFKD",t[i+5].split(" ")[0]).replace(',','.')))
#             except:
#                 SIZE = 0
#             # print(SIZE)
#             # Year
#             try:
#                 YEAR = int(t[i+7].strip())
#             except:
#                 YEAR = 0
#             # print(YEAR)
#             print(" | ".join([str(x) for x in [HOUSE_TYPE,HOUSE_ROOMS,ADDRESS,AREA,CITY,PRICE,SIZE,YEAR]]))
#             # insert_data(conn,HOUSE_TYPE,HOUSE_ROOMS,ADDRESS,AREA,CITY,PRICE,SIZE,YEAR)
#     i+=1

In [ ]:
# Used to create the table from scratch (debugging mostly)
# conn = create_connection(r"../Data/house.db")   
# cursor = conn.cursor()

# cursor.execute("""CREATE TABLE IF NOT EXISTS house_features (
#     ID integer PRIMARY KEY,
#     HOUSE_TYPE text,
#     HOUSE_ROOMS text,
#     ADDRESS text,
#     AREA text,
#     CITY text,
#     PRICE integer,
#     SIZE integer,
#     YEAR integer
# );
# """)
# conn.commit()
# conn.close()

# def delete_all_houses(conn):
#     """
#     Delete all rows in the tasks table
#     :param conn: Connection to the SQLite database
#     :return:
#     """
#     sql = 'DELETE FROM house_features'
#     cur = conn.cursor()
#     cur.execute(sql)
#     conn.commit()
# delete_all_houses(conn)